In [64]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [65]:
dftrain = pd.read_csv('http://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('http://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [66]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERICAL_COLUMNS = ['age', 'fare']
feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    
for feature_name in NUMERICAL_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [74]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) # create tf.data.Dataset object with data and its labels
        if shuffle:
            ds = ds.shuffle(1000) # randomize order of data
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds # return a batch of dataset
    return input_function # return a function object to use

In [75]:
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [76]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
                                           
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

print(result['accuracy'])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Admiral\\AppData\\Local\\Temp\\tmp2uujaql5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32,

In [77]:
print(result)

{'accuracy': 0.7689394, 'accuracy_baseline': 0.625, 'auc': 0.8309764, 'auc_precision_recall': 0.7897891, 'average_loss': 0.48169234, 'label/mean': 0.375, 'loss': 0.4732897, 'precision': 0.7111111, 'prediction/mean': 0.35114694, 'recall': 0.64646465, 'global_step': 200}


In [79]:
rslt = list(linear_est.predict(eval_input_fn))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Admiral\AppData\Local\Temp\tmp2uujaql5\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [119]:
for a in range(len(dfeval)):
    print(dfeval.loc[a])
    print('---------------------------------')
    print('Survival probability: ',rslt[a]['probabilities'][1])
    print('Actual value: ', y_eval[a])
    print('------------------------------------------------------------------------------------------')

sex                          male
age                            35
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 0, dtype: object
---------------------------------
Survival probability:  0.06864076
Actual value:  0
------------------------------------------------------------------------------------------
sex                          male
age                            54
n_siblings_spouses              0
parch                           0
fare                      51.8625
class                       First
deck                            E
embark_town           Southampton
alone                           y
Name: 1, dtype: object
---------------------------------
Survival probability:  0.2654991
Actual value:  0
----------------------------------------------------------------------------------

Actual value:  1
------------------------------------------------------------------------------------------
sex                          male
age                            62
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                      unknown
embark_town           Southampton
alone                           y
Name: 156, dtype: object
---------------------------------
Survival probability:  0.10195702
Actual value:  0
------------------------------------------------------------------------------------------
sex                          male
age                            28
n_siblings_spouses              0
parch                           0
fare                         13.5
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 157, dtype: object
---------------------------------
Survival probability